<a href="https://colab.research.google.com/github/MusabUmama/TSLA_Stock_Price_Prediction/blob/main/Tesla_Stock_Price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd

In [80]:
df = pd.read_csv('/content/tesla_historical_data_jan_to_aug_2023.csv')

df.fillna(df.mean(), inplace=True)

df['Date'] = pd.to_datetime(df['Date'])

def convert_dollar_string_to_number(cell):
    if isinstance(cell, str) and cell.startswith('$'):
        # Remove dollar sign and commas, and convert to float
        return float(cell.replace('$', '').replace(',', ''))
    else:
        return cell

df = df.applymap(convert_dollar_string_to_number)

<ipython-input-80-ff10095c1c95>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [81]:
df['Daily_Return'] = df['Close/Last'].pct_change()

def calculate_moving_average(data, window):
    return data.rolling(window=window, min_periods=1).mean()

df['7_Day_MA'] = calculate_moving_average(df['Close/Last'], window=7)
df['30_Day_MA'] = calculate_moving_average(df['Close/Last'], window=30)

In [82]:
df.dropna(subset=['Daily_Return'], inplace=True)

In [83]:
print(df.head())

        Date  Close/Last     Volume    Open    High     Low  Daily_Return  \
1 2023-08-31      258.08  108861700  255.98  261.18  255.05      0.053345   
2 2023-08-30      256.90  121988400  254.20  260.51  250.59     -0.004572   
3 2023-08-29      257.18  134047600  238.58  257.48  237.77      0.001090   
4 2023-08-28      238.82  107673700  242.58  244.38  235.35     -0.071390   
5 2023-08-25      238.59  106612200  231.31  239.00  230.35     -0.000963   

     7_Day_MA   30_Day_MA  
1  251.545000  251.545000  
2  253.330000  253.330000  
3  254.292500  254.292500  
4  251.198000  251.198000  
5  249.096667  249.096667  


In [84]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [85]:
split_date = '2023-06-01'

train_data = df[df['Date'] < split_date]
test_data = df[df['Date'] >= split_date]

In [86]:
X_train = train_data[['Open', 'High', 'Low', 'Volume', '7_Day_MA', '30_Day_MA']]
y_train = train_data['Daily_Return']
X_test = test_data[['Open', 'High', 'Low', 'Volume', '7_Day_MA', '30_Day_MA']]
y_test = test_data['Daily_Return']

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [88]:
y_pred = model.predict(X_test)

In [89]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

Mean Absolute Error (MAE): 0.0212
Mean Squared Error (MSE): 0.0008
Root Mean Squared Error (RMSE): 0.0279


In [92]:
new_open_value = 250.00
new_high_value = 258.00
new_low_value = 247.00
new_volume_value = 108861700
new_7_day_ma_value = 255.50
new_30_day_ma_value = 253.00


new_data = pd.DataFrame({
    'Open': [new_open_value],
    'High': [new_high_value],
    'Low': [new_low_value],
    'Volume': [new_volume_value],
    '7_Day_MA': [new_7_day_ma_value],
    '30_Day_MA': [new_30_day_ma_value]
})

predicted_return = model.predict(new_data)
print(f'Predicted Daily Return for New Data: {predicted_return[0]:.4f}')

Predicted Daily Return for New Data: 0.0027
